In [15]:
b  = [[]]*2

In [1]:
import argparse
import gc
import logging
import os
import sys
import time

import numpy as np

from collections import defaultdict

import torch
import torch.nn as nn
import torch.optim as optim

from sgan.losses import gan_g_loss, gan_d_loss, l2_loss, l1_loss, length_normalized_l2_loss
from sgan.losses import displacement_error, final_displacement_error

from sgan.various_length_models import TrajectoryDiscriminator, LateAttentionFullGenerator
from sgan.utils import int_tuple, bool_flag, get_total_norm
from sgan.utils import relative_to_abs, get_dset_path
from torch.utils.data import DataLoader

from sgan.data.trajectories import TrajectoryDataset, seq_collate



In [2]:
torch.backends.cudnn.benchmark = True

In [3]:

def get_dtypes(args):
    long_dtype = torch.LongTensor
    float_dtype = torch.FloatTensor
    if args.use_gpu == 1:
        long_dtype = torch.cuda.LongTensor
        float_dtype = torch.cuda.FloatTensor
    return long_dtype, float_dtype

In [4]:

parser = argparse.ArgumentParser()
FORMAT = '[%(levelname)s: %(filename)s: %(lineno)4d]: %(message)s'
logging.basicConfig(level=logging.INFO, format=FORMAT, stream=sys.stdout)
logger = logging.getLogger(__name__)

# Dataset options
parser.add_argument('--dataset_name', default='zara1', type=str)
parser.add_argument('--delim', default=' ')
parser.add_argument('--loader_num_workers', default=4, type=int)
parser.add_argument('--obs_len', default=8, type=int)
parser.add_argument('--pred_len', default=8, type=int)
parser.add_argument('--skip', default=1, type=int)

# Optimization
parser.add_argument('--batch_size', default=64, type=int)
parser.add_argument('--num_iterations', default=10000, type=int)
parser.add_argument('--num_epochs', default=200, type=int)

# Model Options
parser.add_argument('--embedding_dim', default=64, type=int)
parser.add_argument('--num_layers', default=1, type=int)
parser.add_argument('--dropout', default=0, type=float)
parser.add_argument('--batch_norm', default=0, type=bool_flag)
parser.add_argument('--mlp_dim', default=1024, type=int)

# Generator Options
parser.add_argument('--encoder_h_dim_g', default=64, type=int)
parser.add_argument('--decoder_h_dim_g', default=128, type=int)
parser.add_argument('--noise_dim', default=None, type=int_tuple)
parser.add_argument('--noise_type', default='gaussian')
parser.add_argument('--noise_mix_type', default='ped')
parser.add_argument('--clipping_threshold_g', default=0, type=float)
parser.add_argument('--g_learning_rate', default=5e-4, type=float)
parser.add_argument('--g_steps', default=1, type=int)

# Pooling Options
parser.add_argument('--pooling_type', default='pool_net')
parser.add_argument('--pool_every_timestep', default=1, type=bool_flag)

# Pool Net Option
parser.add_argument('--bottleneck_dim', default=1024, type=int)

# Social Pooling Options
parser.add_argument('--neighborhood_size', default=2.0, type=float)
parser.add_argument('--grid_size', default=8, type=int)

# Discriminator Options
parser.add_argument('--d_type', default='local', type=str)
parser.add_argument('--encoder_h_dim_d', default=64, type=int)
parser.add_argument('--d_learning_rate', default=5e-4, type=float)
parser.add_argument('--d_steps', default=2, type=int)
parser.add_argument('--clipping_threshold_d', default=0, type=float)

# Loss Options
parser.add_argument('--intention_loss_weight', default=0, type=float)
parser.add_argument('--intention_loss_type', default='l2', type=str)
parser.add_argument('--l2_loss_weight', default=0, type=float)
parser.add_argument('--resist_loss_weight', default=0, type=float)
parser.add_argument('--heading_loss_weight', default=0, type=float)
parser.add_argument('--best_k', default=1, type=int)

# Output
parser.add_argument('--output_dir', default=os.getcwd())
parser.add_argument('--print_every', default=5, type=int)
parser.add_argument('--checkpoint_every', default=100, type=int)
parser.add_argument('--checkpoint_name', default='checkpoint')
parser.add_argument('--checkpoint_start_from', default=None)
parser.add_argument('--restore_from_checkpoint', default=1, type=int)
parser.add_argument('--num_samples_check', default=5000, type=int)

# Misc
parser.add_argument('--use_gpu', default=1, type=int)
parser.add_argument('--timing', default=0, type=int)
parser.add_argument('--gpu_num', default="0", type=str)
parser.add_argument('--plot_dir', default="../plots/")
parser.add_argument('--benchmark', default=False, type=bool)
parser.add_argument('--spatial_dim', default=True, type=bool)
parser.add_argument('--pruning', default=False, type=bool)

_StoreAction(option_strings=['--pruning'], dest='pruning', nargs=None, const=None, default=False, type=<class 'bool'>, choices=None, help=None, metavar=None)

In [5]:
 args = parser.parse_args("--dataset_name zara1".split())

NameError: name 'goals' is not defined

In [6]:
def data_loader(args, path, shuffle=True, min_ped=1):
    dset = TrajectoryDataset(
        path,
        obs_len=args.obs_len,
        pred_len=args.pred_len,
        skip=args.skip,
        delim='\t', min_ped=min_ped)

    loader = DataLoader(
        dset,
        batch_size=args.batch_size,
        shuffle=shuffle,
        num_workers=args.loader_num_workers,
        collate_fn=seq_collate)
    return dset, loader


In [7]:
dataset_name = 'zara1'
train_path = get_dset_path(dataset_name, 'train')
val_path = get_dset_path(dataset_name, 'val')

long_dtype, float_dtype = get_dtypes(args)

logger.info("Initializing train dataset")
train_dset, train_loader = data_loader(args, train_path)
logger.info("Initializing val dataset")
_, val_loader = data_loader(args, val_path)

[INFO: <ipython-input-7-91b83b450919>:    7]: Initializing train dataset
[INFO: <ipython-input-7-91b83b450919>:    9]: Initializing val dataset


In [95]:
count = 1

for batch in train_loader:
    batch = [tensor.cuda() for tensor in batch]
    (obs_traj, pred_traj_gt, obs_traj_rel, pred_traj_gt_rel, non_linear_ped,
     loss_mask, seq_start_end, goals, goals_rel) = batch
    for start, end in seq_start_end:
        if count==1:
            break
    break

In [20]:
end = end.item()
start = start.item()
end_pos = obs_traj_rel[-1,start:end,:]
num_ped = end-start

In [27]:
    def row_repeat( tensor, num_reps):
        """
        Inputs:
        -tensor: 2D tensor of any shape
        -num_reps: Number of times to repeat each row
        Outpus:
        -repeat_tensor: Repeat each row such that: R1, R1, R2, R2
        """
        col_len = tensor.size(1)
        tensor = tensor.unsqueeze(dim=1).repeat(1, num_reps, 1)
        tensor = tensor.view(-1, col_len)
        return tensor

In [66]:

num_ped = end - start
end_pos = obs_traj_rel[-1, start:end, :]

# r1,r1,r1, r2,r2,r2, r3,r3,r3 - r1,r2,r3, r1,r2,r3, r1,r2,r3
end_pos_difference = row_repeat(end_pos, num_ped) - end_pos.repeat(num_ped, 1)
end_displacement = obs_traj_rel[-1,start:end,:] - obs_traj_rel[-2,start:end,:] / 0.4
end_speed = torch.sqrt(torch.sum(end_displacement**2, dim=1)).view(-1,1)

end_speed_difference =  row_repeat(end_speed, num_ped) - end_speed.repeat(num_ped, 1)
end_heading = torch.atan2(end_displacement[:,0], end_displacement[:,1]).view(-1,1)
end_heading_difference =  row_repeat(end_heading, num_ped) - end_heading.repeat(num_ped, 1)
# num_ped**2
delta_distance = torch.sqrt(torch.sum(end_pos_difference**2, dim=1)).view(-1,1)
# num_ped
delta_speed = torch.abs(end_speed_difference)
delta_heading = torch.abs(torch.atan2(torch.sin(end_heading_difference), torch.cos(end_heading_difference)))



In [94]:
(seq_start_end[:,1] - seq_start_end[:,0]).max()

tensor(57, device='cuda:0')

In [98]:
a = torch.cat((delta_distance, delta_speed, delta_heading),1)
_, _, p = svm_predict([], a.tolist(), m,'-b 1')
p = torch.FloatTensor(p)[:,0]

Accuracy = 0% (0/1156) (classification)


In [96]:
mask = (p>0.5).long().view(num_ped, num_ped)
mask.unsqueeze(2).repeat(1,1,12)

tensor([[[ 1,  1,  1,  ...,  1,  1,  1],
         [ 0,  0,  0,  ...,  0,  0,  0],
         [ 0,  0,  0,  ...,  0,  0,  0],
         ...,
         [ 0,  0,  0,  ...,  0,  0,  0],
         [ 0,  0,  0,  ...,  0,  0,  0],
         [ 1,  1,  1,  ...,  1,  1,  1]],

        [[ 0,  0,  0,  ...,  0,  0,  0],
         [ 1,  1,  1,  ...,  1,  1,  1],
         [ 1,  1,  1,  ...,  1,  1,  1],
         ...,
         [ 0,  0,  0,  ...,  0,  0,  0],
         [ 1,  1,  1,  ...,  1,  1,  1],
         [ 0,  0,  0,  ...,  0,  0,  0]],

        [[ 0,  0,  0,  ...,  0,  0,  0],
         [ 1,  1,  1,  ...,  1,  1,  1],
         [ 1,  1,  1,  ...,  1,  1,  1],
         ...,
         [ 0,  0,  0,  ...,  0,  0,  0],
         [ 1,  1,  1,  ...,  1,  1,  1],
         [ 1,  1,  1,  ...,  1,  1,  1]],

        ...,

        [[ 0,  0,  0,  ...,  0,  0,  0],
         [ 0,  0,  0,  ...,  0,  0,  0],
         [ 0,  0,  0,  ...,  0,  0,  0],
         ...,
         [ 1,  1,  1,  ...,  1,  1,  1],
         [ 0,  0,  0, 

In [14]:
# tracking group related

In [12]:
import os, sys, math, time, copy, numpy
# Parameters
growthAmount = 0.5 / 3.5   # growth of relation strength (between 0.0 and 1.0) per second
slowDecayAmount  = 0.5 / 40.0 # slow decay of relation strength (between 0.0 and 1.0) per second
fastDecayAmount  = 0.5 / 5.0  # fast decay, when maximum distance is exceeded
maxDistance = 4.0  # distance above which relation strength will be forced to 0.0
g_maxDistance = 3.0
g_maxSpeedDifference =1.0
g_maxOrientationDifference = math.pi/4
g_minSpeedToConsiderOrientation=0.1
    

In [ ]:
num_ped = end-start
i=1
for t1Index in range(0, num_ped):
    for t2Index in range(t1Index+1, num_ped):
        p1= obs_traj_rel[i,t1Index, :]
        p2= obs_traj_rel[i,t2Index, :]
        v1 = obs_traj_rel[i,t1Index, :] - obs_traj_rel[i-1,t1Index, :]
        v2 = obs_traj_rel[i,t2Index, :] - obs_traj_rel[i-1,t2Index, :]
        speed1 = torch.sqrt(v1[0]**2 + v1[1]**2) / 0.4
        speed2 = torch.sqrt(v2[0]**2 + v2[1]**2) / 0.4
        theta1 = torch.atan2(v1[0], v1[1]) if speed1 >= g_minSpeedToConsiderOrientation else 0
        theta2 = torch.atan2(v2[0], v2[1]) if speed2 >= g_minSpeedToConsiderOrientation else 0
        distance = torch.sqrt((p1[0] - p2[0])**2 + (p1[1]-p2[1])**2)
        deltaspeed = torch.abs(speed1 - speed2)
        deltaangle = torch.abs(torch.atan2(torch.sin(theta1-theta2), torch.cos(theta1-theta2)))

            
            #Gating for large distance, very different velocities, or very different angle
        if (distance > g_maxDistance or  deltaspeed > g_maxSpeedDifference  or deltaangle > g_maxOrientationDifference):
            
            positiveRelationProbability = 0.1;
            negativeRelationProbability = 0.9;

            
        else:
                # Prepare SVM classifier
                g_svmNode[0].value = distance;
                g_svmNode[1].value = deltaspeed;
                g_svmNode[2].value = deltaangle;

                // Run SVM classifier
                double probabilityEstimates[2];
                svm_predict_probability(g_svmModel, g_svmNode, probabilityEstimates);
                positiveRelationProbability = probabilityEstimates[0];
                negativeRelationProbability = probabilityEstimates[1];
            }

            // Store results for this pair of tracks
            SocialRelation socialRelation;
            socialRelation.type = SocialRelation::TYPE_SPATIAL;
            socialRelation.strength = positiveRelationProbability;
            socialRelation.track1_id = t1.track_id;
            socialRelation.track2_id = t2.track_id;

            socialRelations->elements.push_back(socialRelation);
        }
    }

    // Publish spatial relations
    g_socialRelationsPublisher.publish(socialRelations);
}


        
        

In [52]:
path = '/data/xinjiey/Group_Navi_GAN/env/lib/python3.5/site-packages/libsvm'
sys.path.append(path)
from svmutil import *

In [53]:
m = svm_load_model('//data/xinjiey/Group_Navi_GAN/spencer/group/social_relationships/groups_probabilistic_small.model')

In [54]:
g_svmNode = [svm_node(1,0), svm_node(2,0), svm_node(3,0), svm_node(-1,0)]

In [55]:
a,b,p = svm_predict([], [[0.3,0.4,3.14]], m,'-b 1');

Accuracy = 0% (0/1) (classification)


In [57]:
a

[-1.0]

In [ ]:
# heading related 

In [145]:
print(obs_traj[-1,0,:])
print(obs_traj[0,0,:])
print(obs_traj_rel[-1,0,:])
print(obs_traj_rel[0,0,:])
if(abs(obs_traj_rel[0,0,0] - obs_traj_rel[2,0,0]) <2):
    print("ass")

tensor([6.5707, 1.4210], device='cuda:0')
tensor([5.5327, 1.4601], device='cuda:0')
tensor([ 1.0380, -0.0391], device='cuda:0')
tensor([0., 0.], device='cuda:0')
ass


In [140]:
delta_x = obs_traj_rel[0,:, 0]  - obs_traj_rel[-1,:, 0] 
delta_y = obs_traj_rel[0,:, 1]  - obs_traj_rel[-1,:, 1] 
theta = torch.atan2(delta_x, delta_y)
theta.size()

torch.Size([829])

In [141]:
w = torch.empty(2,2)
nn.init.eye_(w)

tensor([[1., 0.],
        [0., 1.]])

In [245]:
import math
def get_heading_difference(obs_traj_rel, start, end):
    heading_mask = nn.init.eye_(torch.empty(end-start, end-start))
    delta_x = obs_traj_rel[0,start:end, 0]  - obs_traj_rel[-1,start:end, 0] 
    delta_y = obs_traj_rel[0,start:end, 1]  - obs_traj_rel[-1,start:end, 1] 
    theta = torch.atan2(delta_x, delta_y)
    for t in range(0,end-start-1):
        for p in range(t+1, end-start):
            angle = abs(torch.atan2(torch.sin(theta[t]-theta[p]), torch.cos(theta[t]-theta[p])))
            heading_mask[t,p] = heading_mask[p,t] =torch.cos(angle)
    rr = heading_mask.unsqueeze(0).repeat(8,1,1)
    return rr

for start, end in seq_start_end.data:
    heading_mask  = get_heading_difference(obs_traj_rel, start, end).cuda()

    for t in range(start, end):
        if t == start:
            distance = pred_traj_gt[:,t+1:end,:].clone()
            mask  = heading_mask [:, 0, 1:].clone()
        elif t == end-1:
            distance = pred_traj_gt[:,start:t,:].clone()
            mask  = heading_mask [:, -1, :-1].clone()
        else:
            distance = torch.cat((pred_traj_gt[:,start:t,:], pred_traj_gt[:,t+1:end,:]), 1).clone()
            mask = torch.cat((heading_mask[:, t-start, 0:t-start], heading_mask [:,t-start, t+1-start:]), 1).clone() # 8*seq

        distance -= pred_traj_gt[:,t,:].view(-1,1,2)
      
        distance = 0.45 - torch.sqrt(torch.sum(distance**2, dim=2)) - 0.15*mask
        resist_loss = distance[distance > 0.]
        print(resist_loss)

tensor([], device='cuda:0')
tensor([], device='cuda:0')
tensor([], device='cuda:0')
tensor([], device='cuda:0')
tensor([0.0380, 0.0484, 0.0579, 0.0673, 0.0762, 0.0845, 0.0920, 0.0988],
       device='cuda:0')
tensor([], device='cuda:0')
tensor([], device='cuda:0')
tensor([], device='cuda:0')
tensor([0.0380, 0.0484, 0.0579, 0.0673, 0.0762, 0.0845, 0.0920, 0.0988],
       device='cuda:0')
tensor([], device='cuda:0')
tensor([], device='cuda:0')
tensor([], device='cuda:0')
tensor([], device='cuda:0')
tensor([], device='cuda:0')
tensor([], device='cuda:0')
tensor([], device='cuda:0')
tensor([], device='cuda:0')
tensor([], device='cuda:0')
tensor([], device='cuda:0')
tensor([], device='cuda:0')
tensor([], device='cuda:0')
tensor([], device='cuda:0')
tensor([], device='cuda:0')
tensor([], device='cuda:0')
tensor([], device='cuda:0')
tensor([], device='cuda:0')
tensor([], device='cuda:0')
tensor([], device='cuda:0')
tensor([0.0410, 0.0334, 0.0313], device='cuda:0')
tensor([], device='cuda:0')


tensor([], device='cuda:0')
tensor([], device='cuda:0')
tensor([], device='cuda:0')
tensor([], device='cuda:0')
tensor([], device='cuda:0')
tensor([], device='cuda:0')
tensor([], device='cuda:0')
tensor([], device='cuda:0')
tensor([], device='cuda:0')
tensor([], device='cuda:0')
tensor([], device='cuda:0')
tensor([], device='cuda:0')
tensor([], device='cuda:0')
tensor([], device='cuda:0')
tensor([], device='cuda:0')
tensor([], device='cuda:0')
tensor([], device='cuda:0')
tensor([], device='cuda:0')
tensor([], device='cuda:0')
tensor([], device='cuda:0')
tensor([], device='cuda:0')
tensor([], device='cuda:0')
tensor([], device='cuda:0')
tensor([], device='cuda:0')
tensor([], device='cuda:0')
tensor([], device='cuda:0')
tensor([], device='cuda:0')
tensor([], device='cuda:0')
tensor([], device='cuda:0')
tensor([], device='cuda:0')
tensor([], device='cuda:0')
tensor([], device='cuda:0')
tensor([], device='cuda:0')
tensor([], device='cuda:0')
tensor([], device='cuda:0')
tensor([], device='c

tensor([], device='cuda:0')
tensor([], device='cuda:0')
tensor([], device='cuda:0')
tensor([], device='cuda:0')
tensor([], device='cuda:0')
tensor([0.0476, 0.0113, 0.0848], device='cuda:0')
tensor([0.0302, 0.1217], device='cuda:0')
tensor([], device='cuda:0')
tensor([], device='cuda:0')
tensor([], device='cuda:0')
tensor([], device='cuda:0')
tensor([], device='cuda:0')
tensor([], device='cuda:0')
tensor([], device='cuda:0')
tensor([], device='cuda:0')
tensor([], device='cuda:0')
tensor([], device='cuda:0')
tensor([], device='cuda:0')
tensor([], device='cuda:0')
tensor([0.0910], device='cuda:0')
tensor([], device='cuda:0')
tensor([], device='cuda:0')
tensor([], device='cuda:0')
tensor([], device='cuda:0')
tensor([], device='cuda:0')
tensor([], device='cuda:0')
tensor([], device='cuda:0')
tensor([], device='cuda:0')
tensor([], device='cuda:0')
tensor([], device='cuda:0')
tensor([], device='cuda:0')
tensor([], device='cuda:0')
tensor([], device='cuda:0')
tensor([0.0476, 0.0215, 0.0819, 0.

In [239]:
heading_mask = get_heading_difference(obs_traj_rel, 36, 38)

t =0, p=1
torch.Size([2, 2])


In [24]:
pred_traj_gt.size()

torch.Size([8, 689, 2])

In [25]:
obs_traj_rel[-3,:, 0] .size()

torch.Size([689])

In [68]:
obs_traj.view(-1,2).size()

torch.Size([6632, 2])

In [26]:
se = nn.Linear(2,64).cuda()

In [23]:
seq_start_end

tensor([[   0,    3],
        [   3,   30],
        [  30,   53],
        [  53,   55],
        [  55,   59],
        [  59,   61],
        [  61,   65],
        [  65,   68],
        [  68,   95],
        [  95,  105],
        [ 105,  109],
        [ 109,  112],
        [ 112,  117],
        [ 117,  120],
        [ 120,  122],
        [ 122,  164],
        [ 164,  169],
        [ 169,  171],
        [ 171,  176],
        [ 176,  215],
        [ 215,  217],
        [ 217,  227],
        [ 227,  252],
        [ 252,  255],
        [ 255,  265],
        [ 265,  270],
        [ 270,  295],
        [ 295,  298],
        [ 298,  307],
        [ 307,  316],
        [ 316,  323],
        [ 323,  325],
        [ 325,  327],
        [ 327,  366],
        [ 366,  376],
        [ 376,  382],
        [ 382,  403],
        [ 403,  408],
        [ 408,  412],
        [ 412,  416],
        [ 416,  418],
        [ 418,  427],
        [ 427,  432],
        [ 432,  434],
        [ 434,  438],
        [ 

In [27]:
obs_traj_embedding = se(obs_traj.view(-1,2).cuda()).view(-1, obs_traj.size(1), 64)

In [28]:
obs_traj_embedding.size()

torch.Size([8, 689, 64])

In [31]:
def init_hidden( batch=689):
    return (
        torch.zeros(1, batch, 64).cuda(),
        torch.zeros(1, batch, 64).cuda()
    )

In [32]:
encoder = nn.LSTM(64,64, 1, dropout=0.0).cuda()
state_tuple = init_hidden()
output, state = encoder(obs_traj_embedding, state_tuple)

In [37]:
state[0].size()

torch.Size([1, 689, 64])

In [12]:

def make_mlp(dim_list, activation='relu', batch_norm=True, dropout=0):
    layers = []
    for dim_in, dim_out in zip(dim_list[:-1], dim_list[1:]):
        layers.append(nn.Linear(dim_in, dim_out))
        if batch_norm:
            layers.append(nn.BatchNorm1d(dim_out))
        if activation == 'relu':
            layers.append(nn.ReLU())
        elif activation == 'leakyrelu':
            layers.append(nn.LeakyReLU())
        if dropout > 0:
            layers.append(nn.Dropout(p=dropout))
    return nn.Sequential(*layers).cuda()



In [81]:
def get_heading_difference(obs_traj_rel, _start, _end, dim):
    obs_length = 8
    start = _start
    end = _end
    #logger.info('[get_heading_difference]: obs_length is {}, count is {}'.format(obs_traj_rel.size(0), obs_length))
    heading_mask = nn.init.eye_(torch.empty(end-start, end-start))
    delta_x = obs_traj_rel[0,start:end, 0]  - obs_traj_rel[-1,start:end, 0] 
    delta_y = obs_traj_rel[0,start:end, 1]  - obs_traj_rel[-1,start:end, 1] 
    theta = torch.atan2(delta_x, delta_y)
    for t in range(0,end-start-1):
        for p in range(t+1, end-start):
            angle = abs(torch.atan2(torch.sin(theta[t]-theta[p]), torch.cos(theta[t]-theta[p])))
            heading_mask[t,p] = heading_mask[p,t] =torch.cos(angle)
    mask = heading_mask.unsqueeze(2).repeat(1,1,dim).cuda()
    return mask

In [82]:

class PoolHiddenNet(nn.Module):
    """Pooling module as proposed in our paper"""
    def __init__(
        self, embedding_dim=16, h_dim=32, mlp_dim=64, bottleneck_dim=32,
        activation='relu', batch_norm=True, dropout=0.0
    ):
        super(PoolHiddenNet, self).__init__()

        self.mlp_dim = 1024
        self.h_dim = h_dim
        self.bottleneck_dim = bottleneck_dim
        self.embedding_dim = embedding_dim

        mlp_pre_dim = embedding_dim + h_dim
        mlp_pre_pool_dims = [mlp_pre_dim, 512, bottleneck_dim]

        self.spatial_embedding = nn.Linear(2, embedding_dim).cuda()
        self.mlp_pre_pool = make_mlp(
            mlp_pre_pool_dims,
            activation=activation,
            batch_norm=batch_norm,
            dropout=dropout)

    def repeat(self, tensor, num_reps):
        """
        Inputs:
        -tensor: 2D tensor of any shape
        -num_reps: Number of times to repeat each row
        Outpus:
        -repeat_tensor: Repeat each row such that: R1, R1, R2, R2
        """
        col_len = tensor.size(1)
        tensor = tensor.unsqueeze(dim=1).repeat(1, num_reps, 1)
        tensor = tensor.view(-1, col_len)
        return tensor

    def forward(self, h_states, seq_start_end, end_pos):
        """
        Inputs:
        - h_states: Tensor of shape (num_layers, batch, h_dim)
        - seq_start_end: A list of tuples which delimit sequences within batch
        - end_pos: Tensor of shape (batch, 2)
        Output:
        - pool_h: Tensor of shape (batch, bottleneck_dim)
        """
        pool_h = []
        for _, (start, end) in enumerate(seq_start_end):
            start = start.item()
            end = end.item()
            print("start, end:"+str(start)+","+str(end))
            num_ped = end - start
            curr_hidden = h_states.view(-1, self.h_dim)[start:end]
            curr_end_pos = end_pos[start:end]
            # Repeat -> H1, H2, H1, H2
            curr_hidden_1 = curr_hidden.repeat(num_ped, 1)
            # Repeat position -> P1, P2, P1, P2
            curr_end_pos_1 = curr_end_pos.repeat(num_ped, 1)
            # Repeat position -> P1, P1, P2, P2
            curr_end_pos_2 = self.repeat(curr_end_pos, num_ped)
            curr_rel_pos = curr_end_pos_1 - curr_end_pos_2
            mask = get_heading_difference(obs_traj_rel, start, end, self.bottleneck_dim)
            print("curr_rel_pos" + str(curr_rel_pos.size()))
            curr_rel_embedding = self.spatial_embedding(curr_rel_pos)
            print("curr_rel_embedding " + str(curr_rel_embedding.size()))
            mlp_h_input = torch.cat([curr_rel_embedding, curr_hidden_1], dim=1)
            print("mlp_h_input " + str( mlp_h_input.size()))
            curr_pool_h = self.mlp_pre_pool(mlp_h_input)
            print("curr_pool_h " + str( curr_pool_h.size()))
           # print("curr_pool_h.view(num_ped, num_ped, -1).max(1) " + str( curr_pool_h.view(num_ped, num_ped, -1).max(1))
            curr_pool_h = curr_pool_h.view(num_ped, num_ped, -1).mul(mask).max(1)[0]
            print("curr_pool_h " + str( curr_pool_h.size()))
            pool_h.append(curr_pool_h)
        pool_h = torch.cat(pool_h, dim=0)
        return pool_h


In [86]:
torch.ones(3,3,32)

tensor([[[ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
           1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
           1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.],
         [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
           1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
           1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.],
         [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
           1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
           1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.]],

        [[ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
           1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
           1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.],
         [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
           1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
           1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.],
         [ 1.,  1.,  1.,  1.,  1.,  1.,  1.

In [83]:
pool_net = PoolHiddenNet()

In [84]:
get_heading_difference(obs_traj_rel, 0, 3, 32).size()

torch.Size([3, 3, 32])

In [85]:
end_pos = obs_traj[-1, :, :]
force_final_encoder_h = state[0]
pool_h = pool_net(force_final_encoder_h, seq_start_end, end_pos)

start, end:0,3
curr_rel_postorch.Size([9, 2])
curr_rel_embedding torch.Size([9, 16])
mlp_h_input torch.Size([9, 48])
curr_pool_h torch.Size([9, 32])
curr_pool_h torch.Size([3, 32])
start, end:3,30
curr_rel_postorch.Size([729, 2])
curr_rel_embedding torch.Size([729, 16])
mlp_h_input torch.Size([729, 48])
curr_pool_h torch.Size([729, 32])
curr_pool_h torch.Size([27, 32])
start, end:30,53
curr_rel_postorch.Size([529, 2])
curr_rel_embedding torch.Size([529, 16])
mlp_h_input torch.Size([529, 48])
curr_pool_h torch.Size([529, 32])
curr_pool_h torch.Size([23, 32])
start, end:53,55
curr_rel_postorch.Size([4, 2])
curr_rel_embedding torch.Size([4, 16])
mlp_h_input torch.Size([4, 48])
curr_pool_h torch.Size([4, 32])
curr_pool_h torch.Size([2, 32])
start, end:55,59
curr_rel_postorch.Size([16, 2])
curr_rel_embedding torch.Size([16, 16])
mlp_h_input torch.Size([16, 48])
curr_pool_h torch.Size([16, 32])
curr_pool_h torch.Size([4, 32])
start, end:59,61
curr_rel_postorch.Size([4, 2])
curr_rel_embedding

curr_rel_postorch.Size([1089, 2])
curr_rel_embedding torch.Size([1089, 16])
mlp_h_input torch.Size([1089, 48])
curr_pool_h torch.Size([1089, 32])
curr_pool_h torch.Size([33, 32])
start, end:471,473
curr_rel_postorch.Size([4, 2])
curr_rel_embedding torch.Size([4, 16])
mlp_h_input torch.Size([4, 48])
curr_pool_h torch.Size([4, 32])
curr_pool_h torch.Size([2, 32])
start, end:473,476
curr_rel_postorch.Size([9, 2])
curr_rel_embedding torch.Size([9, 16])
mlp_h_input torch.Size([9, 48])
curr_pool_h torch.Size([9, 32])
curr_pool_h torch.Size([3, 32])
start, end:476,478
curr_rel_postorch.Size([4, 2])
curr_rel_embedding torch.Size([4, 16])
mlp_h_input torch.Size([4, 48])
curr_pool_h torch.Size([4, 32])
curr_pool_h torch.Size([2, 32])
start, end:478,506
curr_rel_postorch.Size([784, 2])
curr_rel_embedding torch.Size([784, 16])
mlp_h_input torch.Size([784, 48])
curr_pool_h torch.Size([784, 32])
curr_pool_h torch.Size([28, 32])
start, end:506,520
curr_rel_postorch.Size([196, 2])
curr_rel_embedding t

In [21]:
tmp11 =[ [row[1]] for row in c]

In [23]:
[x+y for x,y in zip(b[1],tmp11)]

[]

In [8]:
file_name = '/home/asus/torch_gpu_ros_3.6/biwi_eth_train.txt'

In [9]:
all_time = np.genfromtxt(file_name, usecols=(0))
userid = np.genfromtxt(file_name, usecols=(1))
x = np.genfromtxt(file_name, usecols=(2))
y = np.genfromtxt(file_name, usecols=(3))
active_peds_id = np.unique(userid)
times = np.unique(all_time)
active_peds_id.sort()
times.sort()
    allY = np.zeros((len(times), len(addresses)))
    allX = np.zeros((len(times), 2))
    strengths={}
    for address, j in zip(addresses, list(range(len(addresses)))):
        ind = np.nonzero(address==macaddress)
        temp_strengths=strength[ind]
        temp_x=x[ind]
        temp_y=y[ind]
        temp_times = all_time[ind]
        for time in temp_times:
            vals = time==temp_times
            if any(vals):
                ind2 = np.nonzero(vals)
                i = np.nonzero(time==times)
                allY[i, j] = temp_strengths[ind2]
                allX[i, 0] = temp_x[ind2]
                allX[i, 1] = temp_y[ind2]
    X = allX[:, :]
Y = allY[:, :]

'780.0\t1.0\t8.46\t3.59\n'

In [ ]:
state_queue.append({'active_peds_id': active_peds_id,
                                 'peds_pos_t': peds_pos_t,
                                 'time_stamp': scanTime})